### Import packages

In [1]:
import numpy as np
import pandas as pd
import datetime
import pickle
import itertools

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import manhattan_distances,pairwise_distances
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier

from metric_learn import NCA

from sklearn.metrics import roc_auc_score

### Read data

In [2]:
X_train = pd.read_csv('X_train.csv')
X_val = pd.read_csv('X_val.csv')
X_test = pd.read_csv('X_test.csv')

y_train = pd.read_csv('y_train.csv')
y_val = pd.read_csv('y_val.csv')

In [3]:
X_train.set_index('unique_id',inplace=True)
X_val.set_index('unique_id',inplace=True)
X_test.set_index('unique_id',inplace=True)
y_train.set_index('unique_id',inplace=True)
y_val.set_index('unique_id',inplace=True)

### Preprocessings

* Imputation

In [4]:
imputer = SimpleImputer(strategy='mean')
imputer.fit(X_train)

X_train_imp = pd.DataFrame(imputer.transform(X_train),columns=X_train.columns,index=X_train.index)
X_val_imp = pd.DataFrame(imputer.transform(X_val),columns=X_val.columns,index=X_val.index)
X_test_imp = pd.DataFrame(imputer.transform(X_test),columns=X_test.columns,index=X_test.index)

* Min-max scaler

In [5]:
mmscaler = MinMaxScaler()
mmscaler.fit(X_train_imp)

X_train_scaled = pd.DataFrame(mmscaler.transform(X_train_imp),columns=X_train_imp.columns,index=X_train_imp.index)
X_val_scaled = pd.DataFrame(mmscaler.transform(X_val_imp),columns=X_val_imp.columns,index=X_val_imp.index)
X_test_scaled = pd.DataFrame(mmscaler.transform(X_test_imp),columns=X_test_imp.columns,index=X_test_imp.index)

### Transformations

* PCA

In [6]:
pca = PCA(n_components=200)
pca.fit(X_train_scaled)

X_train_pca = pd.DataFrame(pca.transform(X_train_scaled),index=X_train_scaled.index)
X_val_pca = pd.DataFrame(pca.transform(X_val_scaled),index=X_val_scaled.index)
X_test_pca = pd.DataFrame(pca.transform(X_test_scaled),index=X_test_scaled.index)
print('Data shape: ',X_train_pca.shape)
print('Total variance covered as percentage: ', pca.explained_variance_ratio_.sum()*100, '%')

Data shape:  (4494, 200)
Total variance covered as percentage:  91.94781913130623 %


* NCA

In [7]:
nca = NCA(n_components=200,random_state=42)

nca.fit(X_train_scaled,y_train)

X_train_nca = pd.DataFrame(nca.transform(X_train_scaled),index=X_train_scaled.index)
X_val_nca = pd.DataFrame(nca.transform(X_val_scaled),index=X_val_scaled.index)
X_test_nca = pd.DataFrame(nca.transform(X_test_scaled),index=X_test_scaled.index)
print('Data shape: ',X_train_nca.shape)
print(X_train_nca.head())

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Data shape:  (4494, 200)
                  0          1          2         3          4          5    \
unique_id                                                                     
4037        64.296350  14.277528 -27.776992 -0.940347   7.263241   7.646964   
7784        56.810068   2.120134   1.971345 -4.339366   2.900180  10.025141   
6232       110.450635   2.037694 -34.229805  3.605538   5.607881  23.188979   
7254        51.100721  24.496071 -24.300606 -6.408237  -5.258204  16.051394   
4336       109.543512  -5.507654 -39.696788  3.020644  11.081889  19.681044   

                6          7         8          9    ...       190       191  \
unique_id                                            ...                       
4037       6.520696   2.543936 -7.419830  -3.365460  ...  2.386538  1.623397   
7784       1.255251 -12.423129 -4.593506 -15.070648  ...  3.520690  2.348803   
6232      -4.739436   4.496132 -8.803378   1.161673  ...  3.732897  1.741796   
7254       5.435710  

#### Save transformed data

In [8]:
X_train_pca.to_csv('X_train_pca.csv')
X_val_pca.to_csv('X_val_pca.csv')
X_test_pca.to_csv('X_test_pca.csv')

X_train_nca.to_csv('X_train_nca.csv')
X_val_nca.to_csv('X_val_nca.csv')
X_test_nca.to_csv('X_test_nca.csv')

### Try Different Classifiers

* First try KNN, but first apply imputation and normalization

In [84]:
imputer = SimpleImputer(strategy='mean')
imputer.fit(X_train)

X_train_imp = pd.DataFrame(imputer.transform(X_train),columns=X_train.columns,index=X_train.index)
X_val_imp = pd.DataFrame(imputer.transform(X_val),columns=X_val.columns,index=X_val.index)
X_test_imp = pd.DataFrame(imputer.transform(X_test),columns=X_test.columns,index=X_test.index)

In [85]:
pickle.dump(imputer, file = open("imputer.pkl", "wb"))

In [86]:
mmscaler = MinMaxScaler()
mmscaler.fit(X_train_imp)

X_train_scaled = pd.DataFrame(mmscaler.transform(X_train_imp),columns=X_train_imp.columns,index=X_train_imp.index)
X_val_scaled = pd.DataFrame(mmscaler.transform(X_val_imp),columns=X_val_imp.columns,index=X_val_imp.index)
X_test_scaled = pd.DataFrame(mmscaler.transform(X_test_imp),columns=X_test_imp.columns,index=X_test_imp.index)

In [87]:
X_train_experienced = X_train[X_train['experience_flag']==1]
y_train_experienced = y_train[y_train.index.isin(list(X_train_experienced.index))]
X_val_experienced = X_val[X_val['experience_flag']==1]
y_val_experienced = y_val[y_val.index.isin(list(X_val_experienced.index))]
X_test_experienced = X_test[X_test['experience_flag']==1]

X_train_cold = X_train[X_train['experience_flag']==0]
y_train_cold = y_train[y_train.index.isin(list(X_train_cold.index))]
X_val_cold = X_val[X_val['experience_flag']==0]
y_val_cold = y_val[y_val.index.isin(list(X_val_cold.index))]
X_test_cold = X_test[X_test['experience_flag']==0]

In [88]:
X_train_experienced_scaled = X_train_scaled[X_train_scaled['experience_flag']==1]
X_val_experienced_scaled = X_val_scaled[X_val_scaled['experience_flag']==1]
X_test_experienced_scaled = X_test_scaled[X_test_scaled['experience_flag']==1]

X_train_cold_scaled = X_train_scaled[X_train_scaled['experience_flag']==0]
X_val_cold_scaled = X_val_scaled[X_val_scaled['experience_flag']==0]
X_test_cold_scaled = X_test_scaled[X_test_scaled['experience_flag']==0]

In [97]:
pickle.dump(mmscaler, file = open("mmscaler.pkl", "wb"))

In [98]:
knn_perf_all = pd.DataFrame()

for n in [5,10,20,50,100]:
    for weight in ['uniform','distance']:

        knn = KNeighborsClassifier(n_neighbors=n,weights=weight)
        knn.fit(X_train_scaled,y_train)

        pred_train = pd.DataFrame(knn.predict(X_train_scaled),columns=['pred'],index=X_train_scaled.index)
        pred_val = pd.DataFrame(knn.predict(X_val_scaled),columns=['pred'],index=X_val_scaled.index)

        pred_train = pd.merge(pred_train,y_train,how='left',left_index=True, right_index=True)
        pred_val = pd.merge(pred_val,y_val,how='left',left_index=True, right_index=True)

        roc_train = roc_auc_score(pred_train['male_label'],pred_train['pred'])
        roc_val = roc_auc_score(pred_val['male_label'],pred_val['pred'])

        perf_tmp = pd.DataFrame({'n_neighbors':[n],'weights':[weight],'Train ROC':[roc_train],'Val ROC':[roc_val]})

        knn_perf_all = pd.concat([knn_perf_all,perf_tmp])

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning:

A column-vector y was passed when a 1d array was 

In [99]:
knn_perf_all.to_csv('knn_perf_all.csv')

In [100]:
knn_perf_splits = pd.DataFrame()

for n in [5,10,20,50,100]:
    for weight in ['uniform','distance']:

        knn_experienced = KNeighborsClassifier(n_neighbors=n,weights=weight)
        knn_experienced.fit(X_train_experienced_scaled,y_train_experienced)

        pred_train_experienced = pd.DataFrame(knn_experienced.predict(X_train_experienced_scaled),columns=['pred'],index=X_train_experienced_scaled.index)
        pred_val_experienced = pd.DataFrame(knn_experienced.predict(X_val_experienced_scaled),columns=['pred'],index=X_val_experienced_scaled.index)

        pred_train_experienced = pd.merge(pred_train_experienced,y_train_experienced,how='left',left_index=True, right_index=True)
        pred_val_experienced = pd.merge(pred_val_experienced,y_val_experienced,how='left',left_index=True, right_index=True)

        roc_train_experienced = roc_auc_score(pred_train_experienced['male_label'],pred_train_experienced['pred'])
        roc_val_experienced = roc_auc_score(pred_val_experienced['male_label'],pred_val_experienced['pred'])

        knn_cold = KNeighborsClassifier(n_neighbors=n,weights=weight)
        knn_cold.fit(X_train_cold_scaled,y_train_cold)

        pred_train_cold = pd.DataFrame(knn_cold.predict(X_train_cold_scaled),columns=['pred'],index=X_train_cold_scaled.index)
        pred_val_cold = pd.DataFrame(knn_cold.predict(X_val_cold_scaled),columns=['pred'],index=X_val_cold_scaled.index)

        pred_train_cold = pd.merge(pred_train_cold,y_train_cold,how='left',left_index=True, right_index=True)
        pred_val_cold = pd.merge(pred_val_cold,y_val_cold,how='left',left_index=True, right_index=True)

        roc_train_cold = roc_auc_score(pred_train_cold['male_label'],pred_train_cold['pred'])
        roc_val_cold = roc_auc_score(pred_val_cold['male_label'],pred_val_cold['pred'])

        pred_train = pd.concat([pred_train_experienced,pred_train_cold])
        pred_val = pd.concat([pred_val_experienced,pred_val_cold])

        roc_train = roc_auc_score(pred_train['male_label'],pred_train['pred'])
        roc_val = roc_auc_score(pred_val['male_label'],pred_val['pred'])

        perf_tmp = pd.DataFrame({'n_neighbors':[n],'weights':[weight],'Train ROC':[roc_train],'Val ROC':[roc_val],
                                'Train ROC Experienced':[roc_train_experienced],'Val ROC Experienced':[roc_val_experienced],
                                'Train ROC Cold':[roc_train_cold],'Val ROC Cold':[roc_val_cold]})

        knn_perf_splits = pd.concat([knn_perf_splits,perf_tmp])

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning:

A column-vector y was passed when a 1d array was 

In [101]:
knn_perf_splits.to_csv('knn_perf_splits.csv')

* Try LightGBM without early stopping

In [102]:
lgbm_perf_all = pd.DataFrame()

for max_d in [-1,2,10]:
    for lr in [0.01,0.1,0.5]:
        for n_est in [100,200]:
            for min_c in [20,50,100]:
                for cols in [0.5,0.8,1]:

                    lgbm = LGBMClassifier(max_depth=max_d,learning_rate=lr,n_estimators=n_est,min_child_samples=min_c,colsample_bytree=cols)
                    lgbm.fit(X_train,y_train)

                    pred_train = pd.DataFrame(lgbm.predict_proba(X_train)[:,1],columns=['pred'],index=X_train.index)
                    pred_val = pd.DataFrame(lgbm.predict_proba(X_val)[:,1],columns=['pred'],index=X_val.index)

                    pred_train = pd.merge(pred_train,y_train,how='left',left_index=True, right_index=True)
                    pred_val = pd.merge(pred_val,y_val,how='left',left_index=True, right_index=True)

                    roc_train = roc_auc_score(pred_train['male_label'],pred_train['pred'])
                    roc_val = roc_auc_score(pred_val['male_label'],pred_val['pred'])

                    perf_tmp = pd.DataFrame({'max_depth':[max_d],'learning_rate':[lr],'n_estimators':[n_est],
                                             'min_child_samples':[min_c],'colsample_bytree':[cols],
                                             'Train ROC':[roc_train],'Val ROC':[roc_val]})

                    lgbm_perf_all = pd.concat([lgbm_perf_all,perf_tmp])


c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of 

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of 

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of 

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of 

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of 

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



In [103]:
lgbm_perf_all.to_csv('lgbm_perf_all.csv')

In [104]:
lgbm_perf_splits = pd.DataFrame()

for max_d in [-1,2,10]:
    for lr in [0.01,0.1,0.5]:
        for n_est in [100,200]:
            for min_c in [20,50,100]:
                for cols in [0.5,0.8,1]:

                    lgbm = LGBMClassifier(max_depth=max_d,learning_rate=lr,n_estimators=n_est,min_child_samples=min_c,colsample_bytree=cols)
                    lgbm.fit(X_train,y_train)

                    pred_train = pd.DataFrame(lgbm.predict_proba(X_train)[:,1],columns=['pred'],index=X_train.index)
                    pred_val = pd.DataFrame(lgbm.predict_proba(X_val)[:,1],columns=['pred'],index=X_val.index)

                    pred_train = pd.merge(pred_train,y_train,how='left',left_index=True, right_index=True)
                    pred_val = pd.merge(pred_val,y_val,how='left',left_index=True, right_index=True)

                    roc_train = roc_auc_score(pred_train['male_label'],pred_train['pred'])
                    roc_val = roc_auc_score(pred_val['male_label'],pred_val['pred'])
                    
                    
                    lgbm_experienced = LGBMClassifier(max_depth=max_d,learning_rate=lr,n_estimators=n_est,min_child_samples=min_c,colsample_bytree=cols)
                    lgbm_experienced.fit(X_train_experienced,y_train_experienced)

                    pred_train_experienced = pd.DataFrame(lgbm_experienced.predict_proba(X_train_experienced)[:,1],columns=['pred'],index=X_train_experienced.index)
                    pred_val_experienced = pd.DataFrame(lgbm_experienced.predict_proba(X_val_experienced)[:,1],columns=['pred'],index=X_val_experienced.index)

                    pred_train_experienced = pd.merge(pred_train_experienced,y_train_experienced,how='left',left_index=True, right_index=True)
                    pred_val_experienced = pd.merge(pred_val_experienced,y_val_experienced,how='left',left_index=True, right_index=True)

                    roc_train_experienced = roc_auc_score(pred_train_experienced['male_label'],pred_train_experienced['pred'])
                    roc_val_experienced = roc_auc_score(pred_val_experienced['male_label'],pred_val_experienced['pred'])

                    lgbm_cold = LGBMClassifier(max_depth=max_d,learning_rate=lr,n_estimators=n_est,min_child_samples=min_c,colsample_bytree=cols)
                    lgbm_cold.fit(X_train_cold,y_train_cold)

                    pred_train_cold = pd.DataFrame(lgbm_cold.predict(X_train_cold),columns=['pred'],index=X_train_cold.index)
                    pred_val_cold = pd.DataFrame(lgbm_cold.predict(X_val_cold),columns=['pred'],index=X_val_cold.index)

                    pred_train_cold = pd.merge(pred_train_cold,y_train_cold,how='left',left_index=True, right_index=True)
                    pred_val_cold = pd.merge(pred_val_cold,y_val_cold,how='left',left_index=True, right_index=True)

                    roc_train_cold = roc_auc_score(pred_train_cold['male_label'],pred_train_cold['pred'])
                    roc_val_cold = roc_auc_score(pred_val_cold['male_label'],pred_val_cold['pred'])

                    pred_train = pd.concat([pred_train_experienced,pred_train_cold])
                    pred_val = pd.concat([pred_val_experienced,pred_val_cold])

                    roc_train = roc_auc_score(pred_train['male_label'],pred_train['pred'])
                    roc_val = roc_auc_score(pred_val['male_label'],pred_val['pred'])

                    perf_tmp = pd.DataFrame({'max_depth':[max_d],'learning_rate':[lr],'n_estimators':[n_est],
                                             'min_child_samples':[min_c],'colsample_bytree':[cols],
                                             'Train ROC':[roc_train],'Val ROC':[roc_val],
                                             'Train ROC Experienced':[roc_train_experienced],'Val ROC Experienced':[roc_val_experienced],
                                             'Train ROC Cold':[roc_train_cold],'Val ROC Cold':[roc_val_cold]})

                    lgbm_perf_splits = pd.concat([lgbm_perf_splits,perf_tmp])


c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of 

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of 

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of 

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of 

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of 

...

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of 

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of 

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of 

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of 

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of 

In [105]:
lgbm_perf_splits.to_csv('lgbm_perf_splits.csv')

* Try LightGBM with early stopping

In [106]:
lgbm_perf_all_early = pd.DataFrame()

for max_d in [-1,2,10]:
    for lr in [0.01,0.1,0.5]:
        for n_est in [100,200]:
            for min_c in [20,50,100]:
                for cols in [0.5,0.8,1]:

                    lgbm = LGBMClassifier(max_depth=max_d,learning_rate=lr,n_estimators=n_est,min_child_samples=min_c,colsample_bytree=cols)
                    lgbm.fit(X_train,y_train,early_stopping_rounds=5,eval_set=[(X_val, y_val)])

                    pred_train = pd.DataFrame(lgbm.predict_proba(X_train)[:,1],columns=['pred'],index=X_train.index)
                    pred_val = pd.DataFrame(lgbm.predict_proba(X_val)[:,1],columns=['pred'],index=X_val.index)

                    pred_train = pd.merge(pred_train,y_train,how='left',left_index=True, right_index=True)
                    pred_val = pd.merge(pred_val,y_val,how='left',left_index=True, right_index=True)

                    roc_train = roc_auc_score(pred_train['male_label'],pred_train['pred'])
                    roc_val = roc_auc_score(pred_val['male_label'],pred_val['pred'])

                    perf_tmp = pd.DataFrame({'max_depth':[max_d],'learning_rate':[lr],'n_estimators':[n_est],
                                             'min_child_samples':[min_c],'colsample_bytree':[cols],
                                             'Train ROC':[roc_train],'Val ROC':[roc_val]})

                    lgbm_perf_all_early = pd.concat([lgbm_perf_all_early,perf_tmp])


c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[1]	valid_0's binary_logloss: 0.640195
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.636948
[3]	valid_0's binary_logloss: 0.633792
[4]	valid_0's binary_logloss: 0.630773
[5]	valid_0's binary_logloss: 0.627994
[6]	valid_0's binary_logloss: 0.625041
[7]	valid_0's binary_logloss: 0.621957
[8]	valid_0's binary_logloss: 0.618928
[9]	valid_0's binary_logloss: 0.615979
[10]	valid_0's binary_logloss: 0.613279
[11]	valid_0's binary_logloss: 0.61049
[12]	valid_0's binary_logloss: 0.607671
[13]	valid_0's binary_logloss: 0.604821
[14]	valid_0's binary_logloss: 0.602114
[15]	valid_0's binary_logloss: 0.599472
[16]	valid_0's binary_logloss: 0.596769
[17]	valid_0's binary_logloss: 0.594118
[18]	valid_0's binary_logloss: 0.591572
[19]	valid_0's binary_logloss: 0.589465
[20]	valid_0's binary_logloss: 0.58704
[21]	valid_0's binary_logloss: 0.584688
[22]	valid_0's binary_logloss: 0.58221
[23]	valid_0's binary_logloss: 0.579977
[24]	valid_0's binary_logloss: 0

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[1]	valid_0's binary_logloss: 0.640006
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.636737
[3]	valid_0's binary_logloss: 0.633443
[4]	valid_0's binary_logloss: 0.630234
[5]	valid_0's binary_logloss: 0.627127
[6]	valid_0's binary_logloss: 0.623999
[7]	valid_0's binary_logloss: 0.620797
[8]	valid_0's binary_logloss: 0.617764
[9]	valid_0's binary_logloss: 0.614787
[10]	valid_0's binary_logloss: 0.61182
[11]	valid_0's binary_logloss: 0.608864
[12]	valid_0's binary_logloss: 0.606201
[13]	valid_0's binary_logloss: 0.603402
[14]	valid_0's binary_logloss: 0.600781
[15]	valid_0's binary_logloss: 0.597996
[16]	valid_0's binary_logloss: 0.595455
[17]	valid_0's binary_logloss: 0.592917
[18]	valid_0's binary_logloss: 0.590409
[19]	valid_0's binary_logloss: 0.587852
[20]	valid_0's binary_logloss: 0.585306
[21]	valid_0's binary_logloss: 0.582751
[22]	valid_0's binary_logloss: 0.580465
[23]	valid_0's binary_logloss: 0.578325
[24]	valid_0's binary_logloss:

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



...

[1]	valid_0's binary_logloss: 0.541329
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.489139
[3]	valid_0's binary_logloss: 0.471746
[4]	valid_0's binary_logloss: 0.469545
[5]	valid_0's binary_logloss: 0.466095
[6]	valid_0's binary_logloss: 0.465176
[7]	valid_0's binary_logloss: 0.467261
[8]	valid_0's binary_logloss: 0.466741
[9]	valid_0's binary_logloss: 0.46505
[10]	valid_0's binary_logloss: 0.470896
[11]	valid_0's binary_logloss: 0.471321
[12]	valid_0's binary_logloss: 0.475375
[13]	valid_0's binary_logloss: 0.482973
[14]	valid_0's binary_logloss: 0.481685
Early stopping, best iteration is:
[9]	valid_0's binary_logloss: 0.46505


c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[1]	valid_0's binary_logloss: 0.532885
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.497894
[3]	valid_0's binary_logloss: 0.480128
[4]	valid_0's binary_logloss: 0.47624
[5]	valid_0's binary_logloss: 0.470733
[6]	valid_0's binary_logloss: 0.465923
[7]	valid_0's binary_logloss: 0.460176
[8]	valid_0's binary_logloss: 0.458404
[9]	valid_0's binary_logloss: 0.462252
[10]	valid_0's binary_logloss: 0.465822
[11]	valid_0's binary_logloss: 0.468367
[12]	valid_0's binary_logloss: 0.476254
[13]	valid_0's binary_logloss: 0.487275
Early stopping, best iteration is:
[8]	valid_0's binary_logloss: 0.458404


c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[1]	valid_0's binary_logloss: 0.527804
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.488671
[3]	valid_0's binary_logloss: 0.470119
[4]	valid_0's binary_logloss: 0.459512
[5]	valid_0's binary_logloss: 0.457349
[6]	valid_0's binary_logloss: 0.455728
[7]	valid_0's binary_logloss: 0.449469
[8]	valid_0's binary_logloss: 0.45338
[9]	valid_0's binary_logloss: 0.457091
[10]	valid_0's binary_logloss: 0.461866
[11]	valid_0's binary_logloss: 0.464989
[12]	valid_0's binary_logloss: 0.471992
Early stopping, best iteration is:
[7]	valid_0's binary_logloss: 0.449469


In [107]:
lgbm_perf_all_early.to_csv('lgbm_perf_all_early.csv')

In [108]:
lgbm_perf_splits_early = pd.DataFrame()

for max_d in [-1,2,10]:
    for lr in [0.01,0.1,0.5]:
        for n_est in [100,200]:
            for min_c in [20,50,100]:
                for cols in [0.5,0.8,1]:
                                       
                    lgbm_experienced = LGBMClassifier(max_depth=max_d,learning_rate=lr,n_estimators=n_est,min_child_samples=min_c,colsample_bytree=cols)
                    lgbm_experienced.fit(X_train_experienced,y_train_experienced,early_stopping_rounds=5,eval_set=[(X_val, y_val)])

                    pred_train_experienced = pd.DataFrame(lgbm_experienced.predict_proba(X_train_experienced)[:,1],columns=['pred'],index=X_train_experienced.index)
                    pred_val_experienced = pd.DataFrame(lgbm_experienced.predict_proba(X_val_experienced)[:,1],columns=['pred'],index=X_val_experienced.index)

                    pred_train_experienced = pd.merge(pred_train_experienced,y_train_experienced,how='left',left_index=True, right_index=True)
                    pred_val_experienced = pd.merge(pred_val_experienced,y_val_experienced,how='left',left_index=True, right_index=True)

                    roc_train_experienced = roc_auc_score(pred_train_experienced['male_label'],pred_train_experienced['pred'])
                    roc_val_experienced = roc_auc_score(pred_val_experienced['male_label'],pred_val_experienced['pred'])

                    lgbm_cold = LGBMClassifier(max_depth=max_d,learning_rate=lr,n_estimators=n_est,min_child_samples=min_c,colsample_bytree=cols)
                    lgbm_cold.fit(X_train_cold,y_train_cold,early_stopping_rounds=5,eval_set=[(X_val, y_val)])

                    pred_train_cold = pd.DataFrame(lgbm_cold.predict(X_train_cold),columns=['pred'],index=X_train_cold.index)
                    pred_val_cold = pd.DataFrame(lgbm_cold.predict(X_val_cold),columns=['pred'],index=X_val_cold.index)

                    pred_train_cold = pd.merge(pred_train_cold,y_train_cold,how='left',left_index=True, right_index=True)
                    pred_val_cold = pd.merge(pred_val_cold,y_val_cold,how='left',left_index=True, right_index=True)

                    roc_train_cold = roc_auc_score(pred_train_cold['male_label'],pred_train_cold['pred'])
                    roc_val_cold = roc_auc_score(pred_val_cold['male_label'],pred_val_cold['pred'])

                    pred_train = pd.concat([pred_train_experienced,pred_train_cold])
                    pred_val = pd.concat([pred_val_experienced,pred_val_cold])

                    roc_train = roc_auc_score(pred_train['male_label'],pred_train['pred'])
                    roc_val = roc_auc_score(pred_val['male_label'],pred_val['pred'])

                    perf_tmp = pd.DataFrame({'max_depth':[max_d],'learning_rate':[lr],'n_estimators':[n_est],
                                             'min_child_samples':[min_c],'colsample_bytree':[cols],
                                             'Train ROC':[roc_train],'Val ROC':[roc_val],
                                             'Train ROC Experienced':[roc_train_experienced],'Val ROC Experienced':[roc_val_experienced],
                                             'Train ROC Cold':[roc_train_cold],'Val ROC Cold':[roc_val_cold]})

                    lgbm_perf_splits_early = pd.concat([lgbm_perf_splits_early,perf_tmp])


c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[1]	valid_0's binary_logloss: 0.657537
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.654053
[3]	valid_0's binary_logloss: 0.650776
[4]	valid_0's binary_logloss: 0.646868
[5]	valid_0's binary_logloss: 0.643407
[6]	valid_0's binary_logloss: 0.639695
[7]	valid_0's binary_logloss: 0.635994
[8]	valid_0's binary_logloss: 0.632569
[9]	valid_0's binary_logloss: 0.629657
[10]	valid_0's binary_logloss: 0.62644
[11]	valid_0's binary_logloss: 0.622893
[12]	valid_0's binary_logloss: 0.619545
[13]	valid_0's binary_logloss: 0.616559
[14]	valid_0's binary_logloss: 0.612752
[15]	valid_0's binary_logloss: 0.6089
[16]	valid_0's binary_logloss: 0.605877
[17]	valid_0's binary_logloss: 0.603156
[18]	valid_0's binary_logloss: 0.600423
[19]	valid_0's binary_logloss: 0.597169
[20]	valid_0's binary_logloss: 0.594647
[21]	valid_0's binary_logloss: 0.592433
[22]	valid_0's binary_logloss: 0.588897
[23]	valid_0's binary_logloss: 0.58667
[24]	valid_0's binary_logloss: 0.

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[1]	valid_0's binary_logloss: 0.658029
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.656273
[3]	valid_0's binary_logloss: 0.653872
[4]	valid_0's binary_logloss: 0.651334
[5]	valid_0's binary_logloss: 0.648984
[6]	valid_0's binary_logloss: 0.647376
[7]	valid_0's binary_logloss: 0.645252
[8]	valid_0's binary_logloss: 0.642774
[9]	valid_0's binary_logloss: 0.640417
[10]	valid_0's binary_logloss: 0.637812
[11]	valid_0's binary_logloss: 0.636552
[12]	valid_0's binary_logloss: 0.634445
[13]	valid_0's binary_logloss: 0.632092
[14]	valid_0's binary_logloss: 0.630447
[15]	valid_0's binary_logloss: 0.629276
[16]	valid_0's binary_logloss: 0.62811
[17]	valid_0's binary_logloss: 0.626142
[18]	valid_0's binary_logloss: 0.624207
[19]	valid_0's binary_logloss: 0.622227
[20]	valid_0's binary_logloss: 0.620469
[21]	valid_0's binary_logloss: 0.619395
[22]	valid_0's binary_logloss: 0.617388
[23]	valid_0's binary_logloss: 0.615912
[24]	valid_0's binary_logloss:

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



...

[1]	valid_0's binary_logloss: 0.568718
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.529722
[3]	valid_0's binary_logloss: 0.513379
[4]	valid_0's binary_logloss: 0.495666
[5]	valid_0's binary_logloss: 0.489421
[6]	valid_0's binary_logloss: 0.494485
[7]	valid_0's binary_logloss: 0.492949
[8]	valid_0's binary_logloss: 0.507786
[9]	valid_0's binary_logloss: 0.501847
[10]	valid_0's binary_logloss: 0.495519
Early stopping, best iteration is:
[5]	valid_0's binary_logloss: 0.489421


c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[1]	valid_0's binary_logloss: 0.554256
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.486667
[3]	valid_0's binary_logloss: 0.473842
[4]	valid_0's binary_logloss: 0.464119
[5]	valid_0's binary_logloss: 0.461368
[6]	valid_0's binary_logloss: 0.472455
[7]	valid_0's binary_logloss: 0.479734
[8]	valid_0's binary_logloss: 0.484401
[9]	valid_0's binary_logloss: 0.483814
[10]	valid_0's binary_logloss: 0.488434
Early stopping, best iteration is:
[5]	valid_0's binary_logloss: 0.461368


c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[1]	valid_0's binary_logloss: 0.575885
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.557889
[3]	valid_0's binary_logloss: 0.540692
[4]	valid_0's binary_logloss: 0.518007
[5]	valid_0's binary_logloss: 0.509444
[6]	valid_0's binary_logloss: 0.512161
[7]	valid_0's binary_logloss: 0.507468
[8]	valid_0's binary_logloss: 0.506537
[9]	valid_0's binary_logloss: 0.506182
[10]	valid_0's binary_logloss: 0.503633
[11]	valid_0's binary_logloss: 0.509879
[12]	valid_0's binary_logloss: 0.513894
[13]	valid_0's binary_logloss: 0.515726
[14]	valid_0's binary_logloss: 0.517942
[15]	valid_0's binary_logloss: 0.522707
Early stopping, best iteration is:
[10]	valid_0's binary_logloss: 0.503633


In [109]:
lgbm_perf_splits_early.to_csv('lgbm_perf_splits_early.csv')